In [1]:
import sys
import os
import cv2
import imutils
from imutils.video import FPS
import numpy as np
from threading import Thread
import time
from tqdm import tqdm

In [2]:
if sys.version_info >= (3, 0):
    from queue import Queue

else:
    from Queue import Queue

In [3]:
class FileVideoStream:
    def __init__(self,path,queueSize=128):
        self.stream = cv2.VideoCapture(path)
        self.stopped = False
        self.Q = Queue(maxsize = queueSize)
        
    def start(self):
        t = Thread(target=self.update,args=())
        t.daemon = True
        t.start()
        return self

    def update(self):
        while True:
            if self.stopped:
                return
            if not self.Q.full():
                (grabbed, frame) = self.stream.read()
            
                if not grabbed:
                    self.stop()
                    return
            
                self.Q.put(frame)

    def read(self):
        return self.Q.get()

    def more(self):
        return self.Q.qsize() > 0

    def stop(self):
        self.stopped = True

In [4]:
data_path = "D:/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence"
save_path = "D:/real-life-violence-situations-dataset/violence_images"

In [5]:
video_list = os.listdir(data_path)
print(video_list)

['V_1.mp4', 'V_10.mp4', 'V_100.mp4', 'V_1000.mp4', 'V_101.mp4', 'V_102.mp4', 'V_103.mp4', 'V_104.mp4', 'V_105.mp4', 'V_106.mp4', 'V_107.mp4', 'V_108.mp4', 'V_109.mp4', 'V_11.mp4', 'V_110.mp4', 'V_111.mp4', 'V_112.mp4', 'V_113.mp4', 'V_114.mp4', 'V_115.mp4', 'V_116.mp4', 'V_117.mp4', 'V_118.mp4', 'V_119.mp4', 'V_12.mp4', 'V_120.mp4', 'V_121.mp4', 'V_122.mp4', 'V_123.mp4', 'V_124.mp4', 'V_125.mp4', 'V_126.mp4', 'V_127.mp4', 'V_128.mp4', 'V_129.mp4', 'V_13.mp4', 'V_130.mp4', 'V_131.mp4', 'V_132.mp4', 'V_133.mp4', 'V_134.mp4', 'V_135.mp4', 'V_136.mp4', 'V_137.mp4', 'V_138.mp4', 'V_139.mp4', 'V_14.mp4', 'V_140.mp4', 'V_141.mp4', 'V_142.mp4', 'V_143.mp4', 'V_144.mp4', 'V_145.mp4', 'V_146.mp4', 'V_147.mp4', 'V_148.mp4', 'V_149.mp4', 'V_15.mp4', 'V_150.mp4', 'V_151.mp4', 'V_152.mp4', 'V_153.mp4', 'V_154.mp4', 'V_155.mp4', 'V_156.mp4', 'V_157.mp4', 'V_158.mp4', 'V_159.mp4', 'V_16.mp4', 'V_160.mp4', 'V_161.mp4', 'V_162.mp4', 'V_163.mp4', 'V_164.mp4', 'V_165.mp4', 'V_166.mp4', 'V_167.mp4', 'V_168

In [6]:
for video in tqdm(video_list):
    video_path = os.path.join(data_path,video)
    name = video.split('.')[0]
    saving_at = os.path.join(save_path,name)
    if not os.path.exists(saving_at):
        os.makedirs(saving_at)
    
    fvs = FileVideoStream(video_path).start()
    time.sleep(1)  #allowing buffer to fill
    
    counter = 0
    while fvs.more():
        frame = fvs.read()
        frame = imutils.resize(frame,width=224,height=224)
        frame_name = os.path.join(saving_at,str(counter)+'.JPG')
        cv2.imwrite(frame_name,frame)
        counter+=1

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:00:34<00:00,  3.63s/it]
